# DEPA Final Project: Extract Transform Load Process

### Author: Caterpillar (CAT) Team - Gerard Almoninia, John Flowers, Joseph Costello

Data: 


In [2]:
# Common imports
import numpy as np
import pandas as pd

# following Python packages needs to be installed 
import pymysql
import xlrd

# To plot visuals
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
from datetime import datetime

In [3]:
now = datetime.now()
fmtnow = now.strftime('%Y-%m-%d %H:%M:%S')
print(now, fmtnow)

2018-11-27 20:41:02.040829 2018-11-27 20:41:02


In [7]:
# step to check for null values
# need to use read_csv because excel is erroring out

input_df=pd.read_csv('CatResult.csv',low_memory=False)

print(      
      "Row & Column counts: ", '\n', input_df.shape
#displays the columns and count of null values for each
       ,'\n', '\n', "Null counts per column: ", '\n', input_df.isnull().sum()
#check dataframe shape to verify the # of rows and columns
#      , '\n', '\n', "First 5 rows: ", '\n', input_df.head(5), '\n', '\n',
)

Row & Column counts:  
 (8393, 94) 
 
 Null counts per column:  
 SURVEY_TYPE                                                                             0
YEAR                                                                                    0
QUARTER                                                                                 0
MONTH                                                                                   0
MARKETING_ORGANIZATION                                                                  0
MARKETING_REGION                                                                        0
MARKETING_DISTRICT                                                                      0
SERIAL_NO                                                                               0
INDUSTRY                                                                              146
MACHINE_DIVISIONS                                                                     146
MACHINE_BUSINESS_DIVISIONS        

In [10]:
#book = xlrd.open_workbook('CAT_CLEAN_V3_SHORT.xlsx')
book = xlrd.open_workbook('CatResult.xlsx')
sheet = book.sheet_by_index(0)

In [25]:
#Error: 'numpy.float64' object has no attribute 'translate' Inserting value to Mysql in Python'
#Soln: Add these lines before making pymysql connection. It adds new encoder of numpy.float64 
#      for product DPU_TGT_2MIS - [73], DPU_TGT_12MIS - [74] 
#      Also added df.iloc[:,73] = float(df.iloc[:,73])
pymysql.converters.encoders[np.float64] = pymysql.converters.escape_float
pymysql.converters.conversions = pymysql.converters.encoders.copy()
pymysql.converters.conversions.update(pymysql.converters.decoders)

# Connect to the database
connection = pymysql.connect(host='localhost'
                             ,user='root'
                             ,password='root'
                             ,db='caterpillar2'
#                             ,charset='utf8', #don't use charset here
                             ,autocommit=True
                            )

cursor = connection.cursor()
cursor.execute("SET NAMES utf8mb4;") #or utf8 or any other charset you want to handle
cursor.execute("SET CHARACTER SET utf8mb4;") #same as above
cursor.execute("SET character_set_connection=utf8mb4;") #same as above
cursor.execute("SET FOREIGN_KEY_CHECKS=0;") #since foreign keys have been defined

0

In [13]:
row_idx, col_idx = 0, 0
print(sheet.cell(row_idx, col_idx), "nrows=", sheet.nrows-1, "ncols=", sheet.ncols-1)

text:'SURVEY_TYPE' nrows= 8393 ncols= 93


In [14]:
# initialization cell by cell - replace empty with nulls (None)
num_cols = sheet.ncols   # Number of columns
for row_idx in range(0, sheet.nrows):    # Iterate through rows
    for col_idx in range(0, num_cols):  # Iterate through columns
        if sheet.cell_type(row_idx, col_idx) == xlrd.XL_CELL_EMPTY:        
            sheet._cell_types[row_idx][col_idx] = xlrd.XL_CELL_NUMBER
            sheet._cell_values[row_idx][col_idx] = None

# will print each cell of the workbook
#for row_idx in range(81259, sheet.nrows):    # Iterate through rows
#    print ('-'*40)
#    print ('Row: %s' % row_idx)   # Print row number
#    for col_idx in range(0, num_cols):  # Iterate through columns
#        cell_obj = sheet.cell(row_idx, col_idx)  # Get cell object by row, col
#        print ('Column: [%s] cell_obj: [%s]' % (col_idx, cell_obj))

In [15]:
#customer table
#INSERT IGNORE will ignore rows with duplicate values and continue execution without creating an error. 
#Any row that contains a duplicate value will not be inserted
query = """INSERT IGNORE INTO customer (customer_id, city, state_name, country_name, country_code, cat_sales_dollars, 
            dealer_language, language_iso_code, marketing_organization, marketing_region, marketing_district, last_update) 
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""

# note that rows and columns start with 0, rows starts at 1 to header in input file.
for r in range(1, sheet.nrows):
        values =  (sheet.cell(r,64).value, 
                   sheet.cell(r,65).value, 
                   sheet.cell(r,66).value, 
                   sheet.cell(r,67).value, 
                   sheet.cell(r,68).value, 
                   sheet.cell(r,69).value, 
                   sheet.cell(r,62).value, 
                   sheet.cell(r,63).value, 
                   sheet.cell(r,4).value, 
                   sheet.cell(r,5).value, 
                   sheet.cell(r,6).value, 
                   fmtnow)
        cursor.execute(query, values)

C:\Users\jcost\Anaconda3\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1062, "Duplicate entry 'f86a4d05-b4d4-4ca7-a110-fd4e6a32422a' for key 'PRIMARY'")
  result = self._query(query)
C:\Users\jcost\Anaconda3\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1062, "Duplicate entry '1413435' for key 'PRIMARY'")
  result = self._query(query)
C:\Users\jcost\Anaconda3\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1062, "Duplicate entry 'a4c965b2-2e34-4216-b1df-e4fd76604d45' for key 'PRIMARY'")
  result = self._query(query)
C:\Users\jcost\Anaconda3\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1062, "Duplicate entry '0f7385f4-4647-43a8-bd1e-cfde52f7885b' for key 'PRIMARY'")
  result = self._query(query)
C:\Users\jcost\Anaconda3\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1062, "Duplicate entry '10b1f392-0fea-4006-8ffd-b5872cbbd879' for key 'PRIMARY'")
  result = self._query(query)
C:\Users\jcost\Anaconda3\Anaconda3

C:\Users\jcost\Anaconda3\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1062, "Duplicate entry '395f0401-42a9-4cb4-a810-e84574948e1b' for key 'PRIMARY'")
  result = self._query(query)
C:\Users\jcost\Anaconda3\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1062, "Duplicate entry 'ed282690-e7d6-4ded-b2f8-31afd54c8ebe' for key 'PRIMARY'")
  result = self._query(query)
C:\Users\jcost\Anaconda3\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1062, "Duplicate entry 'a5538250-48bc-4b7b-86c0-949fb05abeb4' for key 'PRIMARY'")
  result = self._query(query)
C:\Users\jcost\Anaconda3\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1062, "Duplicate entry '40f5698f-6d40-44ea-b0b5-7bfeea628707' for key 'PRIMARY'")
  result = self._query(query)
C:\Users\jcost\Anaconda3\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1062, "Duplicate entry 'c4320ddd-6028-4eda-8e4c-57c7ac1e5641' for key 'PRIMARY'")
  result = self._query(query)
C:\Us

In [27]:
#product table
query_product = """INSERT IGNORE INTO product (serial_no, industry, machine_divisions, machine_business_divisions, product_group, 
            customer_industry_code_description, principle_work_description, market_segment_code_description, 
            market_sub_segment_description, industry_group_description, customer_industry_subsegment_description, mis2_defects, 
            mis12_defects, mis36_defects, dpu_tgt_2mis, dpu_tgt_12mis, eng_mis2_defects, eng_mis12_defects, eng_mis36_defects, 
            hyd_mis2_defects, hyd_mis12_defects, hyd_mis36_defects, col_mis2_defects, col_mis12_defects, col_mis36_defects, 
            lpt_mis2_defects, lpt_mis12_defect, lpt_mis36_defect, upt_mis2_defects, upt_mis12_defects, upt_mis36_defects, 
            muc_mis2_defects, muc_mis12_defects, muc_mis36_defects,
            last_update)  
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, 
            %s)"""

In [28]:
# note that rows and columns start with 0, rows starts at 1 to header in input file.
for r in range(1, sheet.nrows):
        values_product =  (sheet.cell(r,7).value,
                    sheet.cell(r,8).value,
                    sheet.cell(r,9).value,
                    sheet.cell(r,10).value,
                    sheet.cell(r,11).value,
                    sheet.cell(r,12).value,
                    sheet.cell(r,13).value,
                    sheet.cell(r,14).value,
                    sheet.cell(r,15).value,
                    sheet.cell(r,16).value,
                    sheet.cell(r,17).value,
                    sheet.cell(r,70).value,
                    sheet.cell(r,71).value,
                    sheet.cell(r,72).value,
                    sheet.cell(r,73).value,
                    sheet.cell(r,74).value,
                    sheet.cell(r,75).value,
                    sheet.cell(r,76).value,
                    sheet.cell(r,77).value,
                    sheet.cell(r,78).value,
                    sheet.cell(r,79).value,
                    sheet.cell(r,80).value,
                    sheet.cell(r,81).value,
                    sheet.cell(r,82).value,
                    sheet.cell(r,83).value,
                    sheet.cell(r,84).value,
                    sheet.cell(r,85).value,
                    sheet.cell(r,86).value,
                    sheet.cell(r,87).value,
                    sheet.cell(r,88).value,
                    sheet.cell(r,89).value,
                    sheet.cell(r,90).value,
                    sheet.cell(r,91).value,
                    sheet.cell(r,92).value,
                   fmtnow)
        cursor.execute(query_product, values_product)

C:\Users\jcost\Anaconda3\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1062, "Duplicate entry 'RM600251' for key 'PRIMARY'")
  result = self._query(query)
C:\Users\jcost\Anaconda3\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1062, "Duplicate entry 'E2W02838' for key 'PRIMARY'")
  result = self._query(query)
C:\Users\jcost\Anaconda3\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1062, "Duplicate entry 'ZBD10924' for key 'PRIMARY'")
  result = self._query(query)
C:\Users\jcost\Anaconda3\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1062, "Duplicate entry 'WE202339' for key 'PRIMARY'")
  result = self._query(query)
C:\Users\jcost\Anaconda3\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1062, "Duplicate entry 'GAP10677' for key 'PRIMARY'")
  result = self._query(query)
C:\Users\jcost\Anaconda3\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1062, "Duplicate entry 'GAP12323' for key 'PRIMARY'")
  re

In [16]:
#survey table
query_survey = """INSERT IGNORE INTO survey (survey_recordid, serial_no, customer_id,
                    survey_type, year, quarter, month, month_number,
                    customer_loyalty_index_numericvalue, 
                    customer_loyalty_index_rounded_numberset, 
                    customer_loyalty_group_textset, 
                    overall_satisfaction, 
                    repurchase_satisfaction, 
                    recommend_satisfaction, 
                    pe2_relationship_customer_valued_sat, 
                    pe2_machine_utilization_machine_performance_overall_sat, 
                    pe2_technology_machine_technology_available_sat, 
                    pe2_technology_visionlink_use_frequency_textset, 
                    pe2_technology_visionlink_unsubscribed_reason_textset, 
                    pe2_reliability_machine_quality_overall_sat, 
                    pe2_reliability_machine_major_component_reliability_sat, 
                    pe2_reliability_machine_affecting_quality_perception_textset, 
                    pe2_reliability_machine_level_of_uptime_sat, 
                    pe2_reliability_machine_number_of_repairs_required_sat, 
                    pe2_reliability_machine_number_of_repairs_required_numericvalue, 
                    pe2_reliability_machine_unscheduled_repairs_downtime_sat, 
                    pe2_oo_costs_machine_cost_of_operation_overall_sat, 
                    pe2_oo_costs_machine_cost_area_needs_improvement_textset, 
                    pe2_oo_costs_machine_fuel_consumption_sat, 
                    pe2_machine_supp_dlr_overall_value_sat, 
                    pe2_machine_supp_dlr_improvement_areas_textset, 
                    pe2_machine_supp_dlr_improvement_areas_response_time_textset, 
                    pe2_machine_supp_dlr_improvement_areas_cost_textset, 
                    pe2_machine_supp_dlr_improvement_areas_capability_textset, 
                    pe2_machine_supp_dlr_customer_use_percentage_for_service_textset, 
                    pe2_partswarranty_dlr_avail_of_maintenance_parts_sat, 
                    pe2_partswarranty_dlr_avail_of_repair_parts_sat, 
                    pe2_partswarranty_dlr_repair_parts_avail_issue_probe_textset, 
                    pe2_partswarranty_machine_warranty_sat, 
                    pe2_segmentation_company_eqpmt_fleet_machine_numericvalue, 
                    pe2_segmentation_company_eqpmt_fleet_machine_cat_numericvalue, 
                    customer_recommendations_sentiment_polarity, 
                    customer_recommendations_sentiment_score, 
                    customer_recommendations_sentiment, 
                    last_update)  
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
                  , %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
                  , %s,  %s,  %s,  %s
                  , %s)"""

In [17]:
def month_to_number(string):
    m = {'jan': '01', 'feb': '02', 'mar': '03', 'apr': '04', 'may': '05', 'jun': '06', 
         'jul': '07', 'aug': '08', 'sep': '09', 'oct': '10', 'nov': '11', 'dec': '12'
        }
    s = string.strip()[:3].lower()

    try:
        out = m[s]
        return out
    except:
        out = '01'
        return out
#        raise ValueError('Not a month')

In [20]:
print(month_to_number(sheet.cell(8393,3).value), sheet.cell(8393,7).value)


05 RLD00682


In [21]:
# note that rows and columns start with 0, rows starts at 1 to header in input file.
for r in range(1, sheet.nrows):
#        month_number = month_to_number(sheet.cell(76627,3).value)
        
        values_survey = (sheet.cell(r,93).value,
                        sheet.cell(r,7).value,
                        sheet.cell(r,64).value,
                        sheet.cell(r,0).value,
                        sheet.cell(r,1).value,
                        sheet.cell(r,2).value,
                        sheet.cell(r,3).value,
                        month_to_number(sheet.cell(r,3).value),
                        sheet.cell(r,18).value,
                        sheet.cell(r,19).value,
                        sheet.cell(r,20).value,
                        sheet.cell(r,21).value,
                        sheet.cell(r,22).value,
                        sheet.cell(r,23).value,
                        sheet.cell(r,24).value,
                        sheet.cell(r,25).value,
                        sheet.cell(r,27).value,
                        sheet.cell(r,28).value,
                        sheet.cell(r,29).value,
                        sheet.cell(r,30).value,
                        sheet.cell(r,31).value,
                        sheet.cell(r,32).value,
                        sheet.cell(r,34).value,
                        sheet.cell(r,35).value,
                        sheet.cell(r,36).value,
                        sheet.cell(r,37).value,
                        sheet.cell(r,39).value,
                        sheet.cell(r,40).value,
                        sheet.cell(r,41).value,
                        sheet.cell(r,43).value,
                        sheet.cell(r,44).value,
                        sheet.cell(r,45).value,
                        sheet.cell(r,46).value,
                        sheet.cell(r,47).value,
                        sheet.cell(r,48).value,
                        sheet.cell(r,49).value,
                        sheet.cell(r,50).value,
                        sheet.cell(r,51).value,
                        sheet.cell(r,52).value,
                        sheet.cell(r,55).value,
                        sheet.cell(r,56).value,
                        sheet.cell(r,58).value,
                        sheet.cell(r,59).value,
                        sheet.cell(r,61).value,
                        fmtnow)
        cursor.execute(query_survey, values_survey)

In [22]:
print(sheet._cell_values[8393][57], sheet.nrows)

Spare part prices are too high. Compared to the competition, diesel consumption is at the upper limit. 8394


In [23]:
#survey_text table
query_survey_text = """INSERT IGNORE INTO survey_text (survey_recordid, 
            pe2_machine_utilization_machine_perf_overall_sys_improv_opentext, 
            pe2_machine_utilization_machine_perf_overall_sys_improv_englishv, 
            pe2_reliability_machine_comp_affect_qlty_percep_other_opentext, 
            pe2_reliability_machine_comp_affect_qlty_percep_other_englishv, 
            pe2_reliability_machine_repairs_causing_greatest_downt_opentext, 
            pe2_reliability_machine_repairs_causing_greatest_downt_englishv, 
            pe2_oo_costs_machine_primary_work_env_validation_textset, 
            pe2_oo_costs_machine_primary_work_env_validation_englishv, 
            pe2_partswarranty_machine_warranty_improve_probe_opentext, 
            pe2_partswarranty_machine_warranty_improve_probe_englishv, 
            pe2_feedback_cust_machine_sugg_shared_with_cat_opentext, 
            pe2_feedback_cust_machine_sugg_shared_with_cat_englishv, 
            customer_recommendations, 
            customer_recommendations_englishv, 
            customer_recommendations_topics, 
            last_update)  
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s,  %s,  %s,  %s,  %s,  %s,  %s,  %s, 
            %s)"""

# note that columns start with 0 count and there are only rows-1 due to header.
for r in range(1, sheet.nrows):
        values_survey_text =  (
                    sheet.cell(r,93).value,
                    sheet.cell(r,26).value,
                    None,    
                    sheet.cell(r,33).value,
                    None,    
                    sheet.cell(r,38).value,
                    None,    
                    sheet.cell(r,42).value,
                    None,    
                    sheet.cell(r,53).value,
                    None,    
                    sheet.cell(r,54).value,
                    None,    
                    sheet.cell(r,57).value,
                    None,    
                    sheet.cell(r,60).value,
                    fmtnow)
        cursor.execute(query_survey_text, values_survey_text)

C:\Users\jcost\Anaconda3\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1265, "Data truncated for column 'pe2_feedback_cust_machine_sugg_shared_with_cat_opentext' at row 1")
  result = self._query(query)
C:\Users\jcost\Anaconda3\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1265, "Data truncated for column 'pe2_oo_costs_machine_primary_work_env_validation_textset' at row 1")
  result = self._query(query)
C:\Users\jcost\Anaconda3\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1265, "Data truncated for column 'pe2_partswarranty_machine_warranty_improve_probe_opentext' at row 1")
  result = self._query(query)
C:\Users\jcost\Anaconda3\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1265, "Data truncated for column 'customer_recommendations' at row 1")
  result = self._query(query)
C:\Users\jcost\Anaconda3\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1265, "Data truncated for column 'pe2_reliability_machine_repairs

In [29]:
cursor.execute("SET FOREIGN_KEY_CHECKS=1;") #turn on foreign keys check
cursor.close()
connection.commit()
connection.close()
#sheet.close()